# Coursera Capstone Project
## The Battle of Neighborhoods 

## A description of the problem and a discussion of the background

Tourist Attractions and venues in the state of Massachusetts US Data 
Type of data being extracted is json 
Source: https://geodata.lib.berkeley.edu/catalog/MEATOUR95PT
Data Includes: 
* Museum locations
* Attractions
* Shows the Longitude and latitude of each museum and the geometry
* Addresses of museums 
* Analysis the difference in names in each portal name 
* The city of each museum 
* postal code of each tourist attraction 





## What we wish to extract or solve.



Finding the closest nearby museum and tourist attraction close to the main city of Boston

In [6]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [7]:
!wget -q -O 'data.json'  https://geodata.lib.berkeley.edu/download/file/MEATOUR95PT-geojson.json
print('Data downloaded!')

Data downloaded!


In [8]:
with open('data.json') as json_data:
    d_data = json.load(json_data)

In [9]:
d_data

{'type': 'FeatureCollection',
 'totalFeatures': 61,
 'features': [{'type': 'Feature',
   'id': 'MEATOUR95PT.1',
   'geometry': {'type': 'Point', 'coordinates': [-70.87122626, 42.57478635]},
   'geometry_name': 'the_geom',
   'properties': {'ID': 1,
    'MUSEUM_ATT': 'North Shore Music Theatre',
    'ATTRACTION': 'T',
    'STREETADDR': '62 Dunham Rd',
    'COMMUNITY': 'Beverly',
    'ZIPCODE': '01915',
    'TELEPHONE': '508-922-8500',
    'Z996ATTEND': 0,
    'AV_ADD': '62 DUNHAM RD',
    'AV_ZONE': '01915',
    'AV_STATUS': 'M',
    'AV_SCORE': 100,
    'AV_SIDE': 'L',
    'TRAPRANKIN': 0}},
  {'type': 'Feature',
   'id': 'MEATOUR95PT.2',
   'geometry': {'type': 'Point', 'coordinates': [-70.65375957, 42.60561559]},
   'geometry_name': 'the_geom',
   'properties': {'ID': 2,
    'MUSEUM_ATT': 'Gloucester Stage Company',
    'ATTRACTION': 'T',
    'STREETADDR': '267 East Main St',
    'COMMUNITY': 'Gloucester',
    'ZIPCODE': '01930',
    'TELEPHONE': '508-281-4099',
    'Z996ATTEND': 0,


In [10]:
neighborhoods_data = d_data['features']

In [11]:
neighborhoods_data[1]

{'type': 'Feature',
 'id': 'MEATOUR95PT.2',
 'geometry': {'type': 'Point', 'coordinates': [-70.65375957, 42.60561559]},
 'geometry_name': 'the_geom',
 'properties': {'ID': 2,
  'MUSEUM_ATT': 'Gloucester Stage Company',
  'ATTRACTION': 'T',
  'STREETADDR': '267 East Main St',
  'COMMUNITY': 'Gloucester',
  'ZIPCODE': '01930',
  'TELEPHONE': '508-281-4099',
  'Z996ATTEND': 0,
  'AV_ADD': '267 EAST MAIN ST',
  'AV_ZONE': '01930',
  'AV_STATUS': 'M',
  'AV_SCORE': 100,
  'AV_SIDE': 'R',
  'TRAPRANKIN': 0}}

In [12]:
# define the dataframe columns
column_names = [ 'Community','Museum', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [13]:
neighborhoods

,Community,Museum,Latitude,Longitude


In [14]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['MUSEUM_ATT'] 
    neighborhood_name = data['properties']['COMMUNITY']

    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]

    
    neighborhoods = neighborhoods.append({'Museum': borough,
                                          'Community': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    


In [15]:
neighborhoods.head()

,Community,Museum,Latitude,Longitude
0,Beverly,North Shore Music Theatre,42.574786,-70.871226
1,Gloucester,Gloucester Stage Company,42.605616,-70.653760
2,Cohasset,South Shore Music Circus,42.242393,-70.809977
3,Saugus,Saugus Iron Works,42.468261,-71.008434
4,Hingham,Old Ship Church,42.240161,-70.885381


In [16]:
neighborhoods.describe()

,Latitude,Longitude
count,61.000000,61.000000
mean,42.327787,-71.036174
std,0.116313,0.161803
min,42.044226,-71.454325
25%,42.256233,-71.097920
50%,42.352016,-71.056105
75%,42.372161,-70.944222
max,42.605616,-70.653760


In [17]:
neighborhoods.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 4 columns):
Community    61 non-null object
Museum       61 non-null object
Latitude     61 non-null float64
Longitude    61 non-null float64
dtypes: float64(2), object(2)
memory usage: 2.0+ KB


In [18]:
%matplotlib inline 

import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.style.use('ggplot') # optional: for ggplot-like style

# check for latest version of Matplotlib
print ('Matplotlib version: ', mpl.__version__) # >= 2.0.0

Matplotlib version:  3.0.2


In [27]:
neighborhoods.sort_values(['Community'], ascending=False, axis=0, inplace=True)

# get the top 5 entries
top5 = neighborhoods.head()



top5.head()

,Community,Museum,Latitude,Longitude
13,Weymouth,Abigail Adams House,42.231293,-70.944222
35,Stoneham,Stone Zoo,42.464029,-71.093583
19,Scituate,Cudworth House,42.198321,-70.753796
3,Saugus,Saugus Iron Works,42.468261,-71.008434
36,Salem,The House of Seven Gables,42.522262,-70.883927


In [36]:
print('The dataframe has {} communities and {} museums.'.format(
        len(neighborhoods['Museum'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 61 communities and 61 museums.


In [37]:
address = '267 East Main St,01930'

geolocator = Nominatim(user_agent="n_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of The community are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of The community are 42.6149035743491, -70.6586529870203.


In [38]:
# create map of Massachusets using latitude and longitude values
map_mass = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Museum'], neighborhoods['Community']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_mass)  
    
map_mass

In [51]:
#map boston museums
boston_data = neighborhoods[neighborhoods['Museum'] == 'Boston'].reset_index(drop=True)
boston_data.head()

,Community,Museum,Latitude,Longitude


In [52]:
address = '955 Boylston Street,02115'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Boston  955 Boylston Street are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Boston  955 Boylston Street are 42.34805415, -71.085915409633.


In [53]:
# create map of Boston using latitude and longitude values
map_boston = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(boston_data['Latitude'], boston_data['Longitude'], boston_data['Museum']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#c4e518',
        fill_opacity=0.7,
        parse_html=False).add_to(map_boston)  
    
map_boston

In [41]:
CLIENT_ID = '1U5GBTL1KEHWHMMFH25BKFHGD1201XISMRZ04WGMXJQTZAQJ' # your Foursquare ID
CLIENT_SECRET = 'JLAJM1S3JCZUNSAKGOVE34IBSDRLKHWG25PX54QBNV30PS41' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1U5GBTL1KEHWHMMFH25BKFHGD1201XISMRZ04WGMXJQTZAQJ
CLIENT_SECRET:JLAJM1S3JCZUNSAKGOVE34IBSDRLKHWG25PX54QBNV30PS41


In [45]:
neighborhood_latitude = boston_data.loc['Latitude'] # neighbourhood latitude value
neighborhood_longitude = boston_data.loc['Longitude'] # neighbourhood longitude value

neighborhood_name = boston_data.loc[1, 'Museum'] # neighbourhood name

print('Latitude and longitude values of boston"{}" are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

KeyError: 'Latitude'

In [46]:

boston_data.loc[0, 'Community']

KeyError: 0

In [47]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [205]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

NameError: name 'results' is not defined

In [48]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
          # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)